In [1]:
import geopandas as gpd
import pandas as pd
import structlog
import yaml
from shapely import from_wkt


def create_lookup(essential_amenities: dict[str, list[str]]) -> dict[str, str]:
    result = {}
    for cat, osm_list in essential_amenities.items():
        for i in osm_list:
            result[i] = cat
    return result


with open("../data/essential_amenities.yaml") as fp:
    essential_amenities = yaml.safe_load(fp)

logger = structlog.get_logger()

lookup = create_lookup(essential_amenities)

In [2]:
CITY = "madrid"

In [3]:
amenities = gpd.read_file(
    f"../output/{CITY}/amenities/amenities_filtered.wkt.csv", engine="pyogrio"
)
amenities.rename({"category": "osm_category"}, axis=1, inplace=True)
amenities["category"] = amenities["osm_category"].map(lookup)
amenities.dropna(subset=["category"], inplace=True)
amenities.drop(["osm_type", "amenity_type", "amenity_subtype"], axis=1, inplace=True)
amenities["geometry"] = amenities["geometry"].apply(from_wkt)
amenities = gpd.GeoDataFrame(amenities, geometry="geometry", crs=4326)
amenities.head(3)

,osm_id,osm_category,name,geometry_type,geometry,category
32,25931627,amenity:cinema,mk2 Palacio de Hielo,Point,POINT (-3.6361 40.46314),cultural_institutions
36,26065697,amenity:restaurant,Café Comercial,Point,POINT (-3.702 40.42873),restaurants
37,26065699,amenity:restaurant,Honest Greens,Point,POINT (-3.7017 40.42703),restaurants


In [4]:
isochrones = gpd.read_file(f"../output/{CITY}/isochrones.geojson", engine="pyogrio")
isochrones.head(3)

,stop_id,costing,range,geometry
0,5130,walk,5,"POLYGON ((-3.68506 40.51101, -3.68806 40.51031..."
1,5130,walk,10,"POLYGON ((-3.68306 40.513, -3.68606 40.51116, ..."
2,5130,walk,15,"POLYGON ((-3.69006 40.51454, -3.69306 40.51509..."


In [5]:
result = pd.DataFrame()
for i in isochrones[["costing", "range"]].drop_duplicates().itertuples():
    ac = (
        isochrones.query(f"range == {i.range} & costing=='{i.costing}'")
        .sjoin(amenities)
        .groupby(["stop_id", "category"])["osm_id"]
        .count()
        .reset_index()
    )
    ac.rename({"osm_id": "count"}, axis=1, inplace=True)

    ac = (
        pd.pivot_table(ac, index=["stop_id"], columns=["category"], values="count")
        .fillna(0)
        .map(int)
        .reset_index()
    )

    ac["costing"] = i.costing
    ac["range"] = i.range
    result = pd.concat([result, ac])

result.to_csv(f"../output/{CITY}/amenity_counts_in_accessibility.csv", index=False)

In [6]:
sgfw = gpd.read_file(f"../output/{CITY}/stop_geometries_from_walk.geojson")

In [7]:
sgfw.head(3)

,stop_id,area,ellipticity,geometry
0,5130,1.977,0.8803,"MULTIPOLYGON (((-3.69881 40.50731, -3.69878 40..."
1,5166,1.977,0.8803,"MULTIPOLYGON (((-3.69881 40.50731, -3.69878 40..."
2,3881,1.984,0.5615,"MULTIPOLYGON (((-3.68241 40.50059, -3.68501 40..."


In [8]:
temp = sgfw.sjoin(amenities)
temp = temp.groupby(["stop_id", "category"])["osm_id"].count().reset_index()
temp.rename({"osm_id": "count"}, axis=1, inplace=True)
temp = (
    pd.pivot_table(temp, index=["stop_id"], columns=["category"], values="count")
    .fillna(0)
    .map(int)
    .reset_index()
)
temp["costing"] = "public_transport"
# temp["range"] = "10+5"
temp["range"] = pd.NA
temp.to_csv(
    f"../output/{CITY}/amenity_counts_in_public_transport_accessibility.csv",
    index=False,
)